In [1]:
from keras_segmentation.models.segnet import *
import numpy as np
from keras_segmentation.predict import predict_multiple, predict, evaluate
from keras_segmentation.models import all_models
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf

train_images =  "training_set/train_img/"
train_annotations = "training_set/train_ana/"
val_images="training_set/val_img/"
val_annotations="training_set/val_ana/"

In [17]:
model_name = ["vgg_segnet", "resnet50_segnet", "vgg_unet", "resnet50_unet", "vgg_pspnet", "resnet50_pspnet"]
cm = 0
m_version = '_1.1'

model = all_models.model_from_name[model_name[cm]](n_classes=7, input_height=1056, input_width=1440)
# model = all_models.model_from_name[model_name[cm]](n_classes=7, input_height=960, input_width=1344)   ###  This is for pspnet model size


if cm % 2 == 0:
    L_num = 15
    # L_num = 20      ###This is for pspnet model size
else:
    L_num = 141
    # L_num = 175     ###This is for pspnet model size

freeze = True
if freeze:
    m_version = m_version+"_frozen"
    for L in range(L_num):
        model.get_layer(index=L).trainable = False
model.summary(show_trainable=True)

print("Using "+model_name[cm]+m_version+" model")


Model: "model_31"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_8 (InputLayer)        [(None, 1056, 1440, 3)]   0         N          
                                                                            
 block1_conv1 (Conv2D)       (None, 1056, 1440, 64)    1792      N          
                                                                            
 block1_conv2 (Conv2D)       (None, 1056, 1440, 64)    36928     N          
                                                                            
 block1_pool (MaxPooling2D)  (None, 528, 720, 64)      0         N          
                                                                            
 block2_conv1 (Conv2D)       (None, 528, 720, 128)     73856     N          
                                                                            
 block2_conv2 (Conv2D)       (None, 528, 720, 128)     147

In [18]:
### Only for tensorboard callback
# log_dir="logs/fit/"+model_name[cm]+m_version
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoints_path = "./checkpoints/"+model_name[cm]+m_version+"/"+model_name[cm]
out_dir="output_dir/"+model_name[cm]+m_version

model.train(
    train_images =  "training_set/train_img/",
    train_annotations = "training_set/train_ana/",
    validate=True,
    val_images="training_set/val_img/",
    val_annotations="training_set/val_ana/",
    auto_resume_checkpoint=True,
    checkpoints_path = checkpoints_path, 
    epochs=15,
    batch_size=4,
    steps_per_epoch=512,
    val_batch_size=2,
    val_steps_per_epoch=8,
    do_augment=True
)

ev = model.evaluate_segmentation(inp_images_dir=val_images, annotations_dir=val_annotations)
assert ev['frequency_weighted_IU'] > 0.01
print(ev)

more_lines = ['','','', "Model Name:", str(model_name[cm]+m_version),"checkpoints_path: ", checkpoints_path, "model IOU: ", str(ev)]
with open('models_evaluate.txt', 'a') as f:
    f.writelines('\n'.join(more_lines))
    
out_dir="output_dir/"+model_name[cm]+m_version
# checkpoints_path = './checkpoints/resnet50_segnet_1.0(unfrozen)/resnet50_segnet'

predict_multiple(
        model = model,
        inp_dir = val_images, 
        # checkpoints_path = checkpoints_path, 
        out_dir = out_dir+"/nomalSeg")


predict_multiple(
        model = model, 
        inp_dir=val_images, 
        out_dir=out_dir+"/overlay",
        # checkpoints_path=checkpoints_path, 
        overlay_img=True,
        class_names=['_background_', 'normal', 'spiral', 'loose', 'tensioned', 'twisted', 'damage'], 
        show_legends=True)

Verifying training dataset


100%|█████████████████████████████████████████| 185/185 [00:05<00:00, 33.04it/s]


Dataset verified! 
Verifying validation dataset


100%|███████████████████████████████████████████| 75/75 [00:02<00:00, 33.15it/s]


Dataset verified! 
Epoch 1/15
512/512 [==============================] - ETA: 0s - loss: 0.2435 - accuracy: 0.9594
Epoch 1: saving model to ./checkpoints/vgg_segnet_1.1_frozen/vgg_segnet.00001
512/512 [==============================] - 470s 919ms/step - loss: 0.2435 - accuracy: 0.9594 - val_loss: 0.0458 - val_accuracy: 0.9870
Epoch 2/15
512/512 [==============================] - ETA: 0s - loss: 0.0536 - accuracy: 0.9845
Epoch 2: saving model to ./checkpoints/vgg_segnet_1.1_frozen/vgg_segnet.00002
512/512 [==============================] - 493s 965ms/step - loss: 0.0536 - accuracy: 0.9845 - val_loss: 0.0448 - val_accuracy: 0.9876
Epoch 3/15
512/512 [==============================] - ETA: 0s - loss: 0.0498 - accuracy: 0.9854
Epoch 3: saving model to ./checkpoints/vgg_segnet_1.1_frozen/vgg_segnet.00003
512/512 [==============================] - 484s 947ms/step - loss: 0.0498 - accuracy: 0.9854 - val_loss: 0.0395 - val_accuracy: 0.9881
Epoch 4/15
512/512 [==============================] - 

75it [00:14,  5.26it/s]


{'frequency_weighted_IU': 0.9763399952513969, 'mean_IU': 0.40266837945723183, 'class_wise_IU': array([0.98714702, 0.68090104, 0.25769055, 0.07683486, 0.23214338,
       0.43246268, 0.15149912])}


100%|███████████████████████████████████████████| 75/75 [00:18<00:00,  4.15it/s]


[array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
   

In [8]:
out_dir="output_dir/"+model_name[cm]+m_version
# checkpoints_path = './checkpoints/resnet50_segnet_1.0(unfrozen)/resnet50_segnet'

predict_multiple(
        model = model,
        inp_dir = val_images, 
        # checkpoints_path = checkpoints_path, 
        out_dir = out_dir+"/nomalSeg")


predict_multiple(
        model = model, 
        inp_dir=val_images, 
        out_dir=out_dir+"/overlay",
        # checkpoints_path=checkpoints_path, 
        overlay_img=True,
        class_names=['_background_', 'normal', 'spiral', 'loose', 'tensioned', 'twisted', 'damage'], 
        show_legends=True)

100%|███████████████████████████████████████████| 75/75 [00:18<00:00,  4.08it/s]


[array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
   

In [29]:
ev = model.evaluate_segmentation(inp_images_dir=val_images, annotations_dir=val_annotations)
assert ev['frequency_weighted_IU'] > 0.01
print(ev)

more_lines = ['','','', "Model Name:", str(model_name[cm]+m_version),"checkpoints_path: ", checkpoints_path, "model IOU: ", str(ev)]
with open('models_evaluate.txt', 'a') as f:
    f.writelines('\n'.join(more_lines))

75it [00:14,  5.27it/s]

{'frequency_weighted_IU': 0.9826684078435244, 'mean_IU': 0.38516407661811203, 'class_wise_IU': array([0.99199162, 0.74458487, 0.19484083, 0.16267903, 0.25653669,
       0.34551551, 0.        ])}


In [34]:
# checkpoints_path = ''

out = predict(inp="test.png", checkpoints_path = checkpoints_path,
                overlay_img= True, 
                out_fname='test_out.png',
                class_names=['_background_', 'normal', 'spiral', 'loose', 'tensioned', 'twisted', 'damage'], 
                show_legends=True)

loaded weights  ./checkPoints/vgg_segnet_1.0/vgg_segnet.00001


2022-06-20 15:23:14.011480: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [48]:
# %tensorboard --logdir checkpoints/vgg